<a href="https://colab.research.google.com/github/Pratyaksha-047/Baymax_bot/blob/main/Baymax_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
from tensorflow.keras.optimizers import SGD
from tensorflow.python.keras.layers import Dense, Activation, Dropout
from tensorflow.python.keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer

In [2]:
nltk.download('wordnet')
nltk.download('punkt')

lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower())
         for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print(len(documents), "documents")
# classes = intents
print(len(classes), "classes", classes)
# words = all words, vocabulary
print(len(words), "unique lemmatized words", words)


97 documents
23 classes ['angry', 'bullying', 'consistent', 'crisis', 'depression', 'failure_motivation', 'fitness_motivation', 'goodbye', 'greetings', 'guilty', 'hyper_empathy_disorder', 'jealous', 'life', 'loneliness', 'love', 'moral values', 'options', 'self_esteem', 'self_hatred', 'study_motivation', 'success', 'thanks', 'unhappy']
284 unique lemmatized words ["'", "'m", "'s", ',', '-', '.', 'a', 'able', 'about', 'actually', 'advise', 'after', 'afterlife', 'ahhhhh', 'alcohol', 'all', 'always', 'am', 'among', 'an', 'and', 'angry', 'anymore', 'anyone', 'are', 'around', 'ashamed', 'asks', 'attention', 'awesome', 'barely', 'be', 'because', 'become', 'being', 'believe', 'belong', 'best', 'better', 'body', 'box', 'bullied', 'but', 'bye', 'called', 'can', 'capable', 'case', 'change', 'character', 'chatting', 'concentration', 'confidence', 'consistency', 'consistent', 'could', 'crave', 'cup', 'day', 'depression', 'difficult', 'disappointed', 'disorder', 'do', 'doe', 'doing', 'don', 'eating

In [5]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [6]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(
        word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")

Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y),
                 epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
20/20 [==============================] - 0s 2ms/step - loss: 3.1491 - accuracy: 0.0619
Epoch 2/200
20/20 [==============================] - 0s 2ms/step - loss: 3.1255 - accuracy: 0.0722
Epoch 3/200
20/20 [==============================] - 0s 1ms/step - loss: 3.0409 - accuracy: 0.1134
Epoch 4/200
20/20 [==============================] - 0s 1ms/step - loss: 3.0116 - accuracy: 0.1340
Epoch 5/200
20/20 [==============================] - 0s 1ms/step - loss: 2.9619 - accuracy: 0.1237
Epoch 6/200
20/20 [==============================] - 0s 1ms/step - loss: 2.9046 - accuracy: 0.1856
Epoch 7/200
20/20 [==============================] - 0s 1ms/step - loss: 2.8295 - accuracy: 0.2062
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 2.7671 - accuracy: 0.1649
Epoch 9/200
20/20 [==============================] - 0s 2ms/step - loss: 2.7025 - accuracy: 0.3402
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 2.5703 - accuracy: 0.3299
Epoch 11/

### Testing with user given input

In [8]:
from tensorflow.python.keras.models import load_model

In [9]:
model = load_model('chatbot_model.h5')
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [11]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return(np.array(bag))

In [12]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [13]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag'] == tag):
            result = random.choice(i['responses'])
            break
    return result

In [14]:
def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

In [15]:
mssg = input("Enter your message: ")
print(chatbot_response(mssg))

Enter your message: my day was horrible
Cheer up; do not let the small matters in life weigh you down. Don’t mess up your time hating anyone, life is too valuable 
